### From articles "Options for smart investor"

https://dzen.ru/id/5f6b2336d03ed90da6bf9d00


In [3]:
import datetime
import pandas as pd
from chart_studio import plotly

from plotly import graph_objs as go
from plotly.offline import iplot, init_notebook_mode
#import plotly.express as px

from option_lib.provider import PandasLocalFileProvider, RequestParameters
from option_lib.entities import OptionType, OptionMoneyStatus
from option_lib import Option


init_notebook_mode(connected=True)

pd.set_option("display.max_rows", 20, "display.max_columns", 30)

In [4]:
cme_provider = PandasLocalFileProvider('CME', '../data')
cur_dt = datetime.date.today()
provider_params = RequestParameters(period_to=cur_dt.year)
brn = Option(cme_provider, 'BRN', provider_params)

In [5]:
df_opt = brn.df_opt
df_opt.head(2)

,datetime,expiration_date,strike,type,premium,future_expiration_date
0,2024-12-17,2024-12-23,250.0,c,7.243377e-84,2024-12-30
1,2024-12-17,2024-12-23,250.0,p,1.768100e+02,2024-12-30


In [6]:
df_fut = brn.df_fut
df_fut.head(2)

,datetime,expiration_date,price
0,2024-12-17,2024-12-30,73.19
1,2024-12-17,2025-01-31,72.85


In [7]:
brn.enrichment.add_future().add_intrinsic_and_time_value()
brn.df_opt.head(2)

,datetime,expiration_date,strike,type,premium,future_expiration_date,future_price,intrinsic_value,time_value
0,2024-12-17,2024-12-23,250.0,c,7.243377e-84,2024-12-30,73.19,0.00,7.243377e-84
1,2024-12-17,2024-12-23,250.0,p,1.768100e+02,2024-12-30,73.19,176.81,-2.441406e-06


In [9]:
df_opt['money_status'] = brn.enrichment.get_atm_itm_otm(df_opt)
df_opt[df_opt['money_status']==OptionMoneyStatus.ATM.code].head(2)

AttributeError: 'OptionEnrichment' object has no attribute 'get_atm_itm_otm'

In [ ]:
settlement_date = df_opt['datetime'].max()
df_opt_date = df_opt[df_opt['datetime']==settlement_date]
list(df_opt_date['expiration_date'].unique())[:5]

In [ ]:
expiration_date = df_opt_date['expiration_date'].min()
settlement_date, expiration_date

In [ ]:
df_opt_chain = df_opt_date[df_opt_date['expiration_date']==expiration_date]
df_opt_chain.head(2)

Типы фьючерсов:
* Валютные
* Индексные
* Товарные
* Фондовые
* Процентные

  Криптовалютные?? отдельный тип или это валютные?


Спецификация WTI https://www.cmegroup.com/markets/energy/crude-oil/light-sweet-crude.html
https://www.investing.com/commodities/crude-oil - график

In [ ]:
atm_nearest_strikes = enrichment.chain.get_chain_atm_nearest_strikes(df_opt_chain)
atm_strike = atm_nearest_strikes[0]
atm_strike

In [ ]:
# options chain / desk
df_opt_desk = enrichment.chain.convert_chain_to_desk(df_opt_chain, ['premium', 'iv' , 'delta', 'gamma', 'vega', 'theta', 'quick_delta', 'money_status', 'intrinsic_value', 'time_value', 'datetime', 'expiration_date', 'strike'])
res_col = ['premium_call', 'iv_call', 'delta_call', 'gamma_call', 'vega_call', 'theta_call', 'quick_delta_call', 'intrinsic_value_call', 'time_value_call', 'money_status_call', 'strike', 'money_status_put','premium_put', 'iv_put', 'delta_put', 'gamma_put', 'vega_put', 'theta_put', 'quick_delta_put', 'intrinsic_value_put', 'time_value_put', 'datetime', 'underlying_price', 'expiration_date', 'underlying_expiration_date']
df_opt_desk [df_opt_desk ['strike'].isin(atm_nearest_strikes[:10])][res_col]

Временная стоимость опциона уменьшается к дате экспирации

Опционы ITM имеют малую величину временной стоимости (малый вес) в общей цене опциона. По сути, чем глубже опцион в деньгах, тем меньше его временная стоимость и тем больше он походит на линейный базовый актив (т.е соответствующий фьючерс).

Опционы OTM не имеют внутренней стоимости (что понятно – их невыгодно исполнять). Вся стоимость внеденежного опциона – это внутренняя стоимость. Важно понимать, что, если рынок не преодолеет страйк этого опциона, то итоговая цена опциона на экспирацию будет равна нулю.

Опционы ATM имеют максимальный вес временной стоимости в цене опциона и максимальную нелинейность. По этой причине являются наиболее ликвидными на рынке. При движении БА вверх или вниз от текущих уровней ликвидность данных опционов будет снижаться.

In [ ]:
df_time_value_strike = df_opt[(df_opt['expiration_date']==expiration_date)&(df_opt['strike']==atm_strike)&(df_opt['type']==OptionType.CALL.code)].sort_values(by='datetime')[['datetime', 'time_value']]
df_time_value_strike.iloc[[0,50,100]]
df_time_value_atm = df_opt[(df_opt['expiration_date']==expiration_date)&(df_opt['money_status']==OptionMoneyStatus.ATM.code)&(df_opt['type']==OptionType.CALL.code)].sort_values(by='datetime')[['datetime', 'time_value']]
# df_time_value_atm.iloc[[0,50,100]]

In [ ]:
# https://plotly.com/python/line-and-scatter/
# Alternative
#fig = px.scatter(df_time_value, x="datetime", y="time_value")
#fig.show()
data = [go.Scatter(x=df_time_value_strike['datetime'].to_list(), y=df_time_value_strike['time_value'].to_list(),mode='lines'),
        go.Scatter(x=df_time_value_atm['datetime'].to_list(), y=df_time_value_atm['time_value'].to_list(),mode='lines')
       ] # markers, lines
iplot(data)

## Нелинейность опционов

In [ ]:
atm_strike_df = df_opt_chain[df_opt_chain['strike'] == atm_strike]
atm_strike_row = atm_strike_df[atm_strike_df['type']==OptionType.CALL.code].iloc[0]
atm_strike_df

In [ ]:
future_price = atm_strike_row['underlying_price']
option_price = atm_strike_row['premium']
contract_size = atm_strike_row['contact_size']
# atm_strike = atm_strike_row['strike']
future_price, option_price, atm_strike

In [ ]:
prices = [price for price in range(int(future_price-25), int(future_price + 25))]
df_trades = pd.DataFrame({'underlying_price': prices, 'future_pnl': [price * contract_size] for price in prices], 'option_pnl': []})
df_trades

In [ ]:
data = [go.Scatter(x=df_trades['underlying_price'].to_list(), y=df_trades['future_pnl'].to_list(),mode='lines'),
        go.Scatter(x=df_trades['underlying_price'].to_list(), y=df_trades['option_pnl'].to_list(),mode='lines')
       ] # markers, lines
iplot(data)